In [1]:
import pandas as pd
from textblob import TextBlob
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

[nltk_data] Downloading package vader_lexicon to /Users/dminzi-
[nltk_data]     lt/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
df = pd.read_csv('../what_goes_in/whatGoesIn.csv')

In [3]:
def flag_prnt(row):
    if row['body'][:7] == "PARENT:":
        return 1
    return 0

def remove_prnt_tag(row):
    if row['body'][:7] == "PARENT:":
        return row['body'][8:]      # intentional removes extra space
    return row['body']

In [4]:
df['pres_body'] = df.apply(remove_prnt_tag, axis=1)

In [5]:
def clean_body(row):
    return re.sub("[^a-zA-Z \' ' \. \,]+", "", row['pres_body']).lower()

In [6]:
df['clean_body'] = df.apply(clean_body, axis=1)

In [7]:
TextBlob(df['clean_body'][1]).sentiment.polarity

0.075

In [8]:
def tb_sentiment(row):
    tb_res = TextBlob(row['clean_body'])

    if tb_res.sentiment.polarity > 0.05:
        return 1
    elif tb_res.sentiment.polarity < 0:
        return -1

    return 0


In [9]:
df['tb_sentiment'] = df.apply(tb_sentiment, axis=1)

In [10]:
df[df.tb_sentiment == 1][df.clean_body.str.len() > 70].sort_values('comment_upvotes', ascending=False).head(2)

,Unnamed: 0,parent_url,parent_upvotes,comment_upvotes,_date,subreddit,body,pres_body,clean_body,tb_sentiment
43326,43326,https://reddit.com/r/CrohnsDisease/comments/kp...,828,823,2021-01-03 19:10:01,CrohnsDisease,PARENT: * It hits everyone differently. For ea...,* It hits everyone differently. For each perso...,it hits everyone differently. for each person...,1
36485,36485,https://reddit.com/r/CrohnsDisease/comments/kd...,547,551,2020-12-15 17:57:24,CrohnsDisease,PARENT: I'm a nurse on a covid unit and am get...,I'm a nurse on a covid unit and am getting the...,i'm a nurse on a covid unit and am getting the...,1


In [11]:
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(df['clean_body'][29])


def vader_sentiment(row):
    vader_res = sia.polarity_scores(row['clean_body'])
    
    if vader_res['pos'] > 0.3:
        return 1
    elif vader_res['neg'] > 0.01:
        return -1
    
    return 0

In [12]:
df['vader_sentiment'] = df.apply(vader_sentiment, axis=1)

In [22]:
for sr in df.subreddit.unique():
    df[df.vader_sentiment == 1][df.tb_sentiment == 1][df.clean_body.str.len() > 40][df.subreddit == sr].sort_values('comment_upvotes', ascending=False).head(20).to_csv(f'{sr}_top20.csv')

In [18]:
df.subreddit.unique()

array(['ibs', 'UlcerativeColitis', 'CrohnsDisease', 'Celiac'],
      dtype=object)